In [1]:
import sys, os
import requests #(네이버) api 이용할 때 쓰는 패키지
import selenium #크롤링 패키지

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup #크롤링 패키지
import re #정규식 표현 모듈
from datetime import datetime
import pickle, json, glob, time #pickle: 객체를 파일로 저장해주는 모듈, json: 네이버 api, glob: 파일들 리스트 뽑을 때 사용
# https://wikidocs.net/83
from tqdm import tqdm #진행표시바


import urllib.request #파싱을 위한 모듈

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np

In [2]:
# ! pip install URI

In [3]:
from uri import URI

# [1] Use This for CIDs

In [222]:
# 각 오퍼레이션 별로 워드에서 검색했을 때, table의 항목구분 열에 1이라고 표시돼있는 필수 파라미터를 다 작성하지 않을 경우 soup 출력시 에러남.

API_HOST = 'http://apis.data.go.kr/B551011/KorService/'
key = '키키키키키키키키키키키키키키키키' # Encoding key

## 오퍼레이션 담은 dic
op = {}

## p14
op['area'] = 'areaCode' # 지역코드조회
# url = f'{API_HOST}{area}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=appTest'


op['det'] = 'detailCommon' # 공통정보조회
# CID = 126508
## p81, content id가 ~~인 관광정보의 '기본정보'만 조회
# url = f'{API_HOST}{det}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&contentId={CID}&defaultYN=Y'


## 오퍼레이션
op['cat'] = 'categoryCode' # 서비스분류코드조회
op['aB'] = 'areaBasedList' # 지역기반관광정보조회


## 파라미터
# CTI = 12
tp = 'json'


url = f'{API_HOST}{op["aB"]}?serviceKey={key}&numOfRows=40000&pageNo=1&MobileOS=ETC&MobileApp=AppTest&listYN=Y'

# &contentTypeId={CTI}
# &_type={tp}
response = requests.get(url)

soup = BeautifulSoup(response.text, 'xml') # parser는 'lxml'로도 가능.

In [226]:
len(soup.find_all('contenttypeid'))

34035

In [264]:
CIDs = soup.find_all('contentid')
len(CIDs) # 하루 지나서 더 늘은 것 같음.
len(set(CIDs)) # 여전히 유니크함.

CIDs = [i.string for i in CIDs]
CIDs = list(map(lambda x: str(x), CIDs)) # str로 형변환

34035

34035

# cid 저장

In [265]:
cids = pd.DataFrame(CIDs, columns=['cid'])
# cids.to_csv('cids.csv', index=False) # str로 형변환, 정렬 안함(contenttypeid랑 엮어야 해서), 업데이트 됨.

In [232]:
soup.find('item')

<item>
<addr1>충청남도 공주시 감영길 3</addr1>
<addr2>(반죽동)</addr2>
<areacode>34</areacode>
<booktour/>
<cat1>A04</cat1>
<cat2>A0401</cat2>
<cat3>A04010600</cat3>
<contentid>2750144</contentid>
<contenttypeid>38</contenttypeid>
<createdtime>20210928012320</createdtime>
<firstimage/>
<firstimage2/>
<mapx>127.1216721795</mapx>
<mapy>36.4529297875</mapy>
<mlevel>6</mlevel>
<modifiedtime>20210928012320</modifiedtime>
<readcount>7</readcount>
<sigungucode>1</sigungucode>
<tel/>
<title>가가상점</title>
<zipcode>32546</zipcode>
</item>

In [271]:
title = [i.string for i in soup.find_all('title')]
title = list(map(lambda x: str(x), title)) # str로 형변환

## 매칭테스트
- 처음에 받은 csv에서 poi id가 0으로 시작하는게 cid에 있는지 찾아봄. -> 없음.
- 처음에 받은 csv에서 poi id가 0으로 시작하는것 의 poi nm이 title에 있는지 찾아봄. -> 없음.

- 처음에 받은 csv에서 poi id가 0으로 시작하지 않는 다른 것의 poi nm이 title에 있는지 찾아봄. -> 있음.
    - 그것의 poi id와 cid가 일치하는지 확인해봄. -> 일치.

In [270]:
for i in CIDs:
#     type(i)
#     break
    if i[0] =='0':
        print('y')
# [i.startswith('0') for i in CIDs]

# 에러 'NoneType' object has no attribute 'startswith'
# 굳이 startswith을 사용하지는 않기로 한다.
# 에러 원인은 value의 type이 str이 아니라 bs4.element.NavigableString 였기 때문

In [277]:
'봉래산' in title
title.index('봉래산')
CIDs[12078] # 처음에 받은 csv 내 봉래산 poi id 와 일치

True

12078

'127810'

# [2] cids로 각각 불러와서 csv파일 만들기

## i) catinfo.csv 만들기
catcd2nm을 용이하게 하기 위한 작업

In [4]:
# 각 오퍼레이션 별로 워드에서 검색했을 때, table의 항목구분 열에 1이라고 표시돼있는 필수 파라미터를 다 작성하지 않을 경우 soup 출력시 에러남.

API_HOST = 'http://apis.data.go.kr/B551011/KorService/'
key = '키키키키키키키키키키키키키키키키' # Encoding key

## 오퍼레이션 담은 dic
op = {}
op['det'] = 'detailCommon' # 공통정보조회
op['cat'] = 'categoryCode' # catcd name 조회

### cat1_cd를 받아서 그 안 cat 범위인 cat2 조회에 사용

In [8]:
# cat_cd를 name으로 바꾸기 위한 조회 (어차피 모든 cat 다 가져옴)
url_cat = f'{API_HOST}{op["cat"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&cat1&cat2&cat3'
response_cat = requests.get(url_cat)
soup_cat = BeautifulSoup(response_cat.text, 'xml') # parser는 'lxml'로도 가능.

In [9]:
cat1_cd_lst = [str(i.string) for i in soup_cat.find_all('code')]
cat1_nm_lst = [str(i.string) for i in soup_cat.find_all('name')]

cat1_cd2nm = {cat1_cd_lst[i]:cat1_nm_lst[i] for i in range(len(cat1_cd_lst))}
cat1_cd2nm

{'A01': '자연',
 'A02': '인문(문화/예술/역사)',
 'A03': '레포츠',
 'A04': '쇼핑',
 'A05': '음식',
 'B02': '숙박',
 'C01': '추천코스'}

In [25]:
cat1_cd2nm = {'A01': '자연',
              'A02': '인문(문화/예술/역사)',
              'A03': '레포츠',
              'A04': '쇼핑',
              'A05': '음식',
              'B02': '숙박',
              'C01': '추천코스'}

### 중분류 cat2

In [6]:
cat2_cd2nm = {}

for i in tqdm(cat1_cd2nm.keys()):
#     cat1_cd2nm[i]
    
    # cat_cd를 name으로 바꾸기 위한 조회 (어차피 모든 cat 다 가져옴)
    url_cat2 = f'{API_HOST}{op["cat"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&cat1={i}&cat2&cat3'
    response_cat2 = requests.get(url_cat2)
    soup_cat2 = BeautifulSoup(response_cat2.text, 'xml') # parser는 'lxml'로도 가능.
#     soup_cat2
    
    cat2_cd_lst = [str(i.string) for i in soup_cat2.find_all('code')]
    cat2_nm_lst = [str(i.string) for i in soup_cat2.find_all('name')]
    cat2_cd2nm[i] = {cat2_cd_lst[j]:cat2_nm_lst[j] for j in range(len(cat2_cd_lst))}
#     break

cat2_cd2nm

100%|██████████| 7/7 [01:55<00:00, 16.46s/it]


{'A01': {'A0101': '자연관광지', 'A0102': '관광자원'},
 'A02': {'A0201': '역사관광지',
  'A0202': '휴양관광지',
  'A0203': '체험관광지',
  'A0204': '산업관광지',
  'A0205': '건축/조형물',
  'A0206': '문화시설',
  'A0207': '축제',
  'A0208': '공연/행사'},
 'A03': {},
 'A04': {'A0401': '쇼핑'},
 'A05': {},
 'B02': {'B0201': '숙박시설'},
 'C01': {'C0112': '가족코스',
  'C0113': '나홀로코스',
  'C0114': '힐링코스',
  'C0115': '도보코스',
  'C0116': '캠핑코스',
  'C0117': '맛코스'}}

In [5]:
cat2_cd2nm = {'A01': {'A0101': '자연관광지', 'A0102': '관광자원'},
              'A02': {'A0201': '역사관광지',
                      'A0202': '휴양관광지',
                      'A0203': '체험관광지',
                      'A0204': '산업관광지',
                      'A0205': '건축/조형물',
                      'A0206': '문화시설',
                      'A0207': '축제',
                      'A0208': '공연/행사'},
              'A03': {},
              'A04': {'A0401': '쇼핑'},
              'A05': {},
              'B02': {'B0201': '숙박시설'},
              'C01': {'C0112': '가족코스',
                      'C0113': '나홀로코스',
                      'C0114': '힐링코스',
                      'C0115': '도보코스',
                      'C0116': '캠핑코스',
                      'C0117': '맛코스'}}

{}

### 소분류 cat3

In [6]:
cat3_cd2nm = {}

for i in tqdm(cat2_cd2nm.keys()):

    for k in cat2_cd2nm[i].keys():
        # cat_cd를 name으로 바꾸기 위한 조회 (어차피 모든 cat 다 가져옴)
        url_cat3 = f'{API_HOST}{op["cat"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&cat1={i}&cat2={k}&cat3'
        response_cat3 = requests.get(url_cat3)
        soup_cat3 = BeautifulSoup(response_cat3.text, 'xml') # parser는 'lxml'로도 가능.

        cat3_cd_lst = [str(i.string) for i in soup_cat3.find_all('code')]
        cat3_nm_lst = [str(i.string) for i in soup_cat3.find_all('name')]
        cat3_cd2nm[f'{i}_{k}'] = {cat3_cd_lst[j]:cat3_nm_lst[j] for j in range(len(cat3_cd_lst))}

cat3_cd2nm

100%|██████████| 7/7 [06:25<00:00, 55.04s/it] 


{'A01_A0101': {},
 'A01_A0102': {},
 'A02_A0201': {'A02010100': '고궁',
  'A02010200': '성',
  'A02010300': '문',
  'A02010400': '고택',
  'A02010500': '생가',
  'A02010600': '민속마을',
  'A02010700': '유적지/사적지',
  'A02010800': '사찰',
  'A02010900': '종교성지',
  'A02011000': '안보관광'},
 'A02_A0202': {},
 'A02_A0203': {'A02030100': '농.산.어촌 체험',
  'A02030200': '전통체험',
  'A02030300': '산사체험',
  'A02030400': '이색체험',
  'A02030500': '관광농원',
  'A02030600': '이색거리'},
 'A02_A0204': {},
 'A02_A0205': {'A02050100': '다리/대교',
  'A02050200': '기념탑/기념비/전망대',
  'A02050300': '분수',
  'A02050400': '동상',
  'A02050500': '터널',
  'A02050600': '유명건물'},
 'A02_A0206': {'A02060100': '박물관',
  'A02060200': '기념관',
  'A02060300': '전시관',
  'A02060400': '컨벤션센터',
  'A02060500': '미술관/화랑',
  'A02060600': '공연장',
  'A02060700': '문화원',
  'A02060800': '외국문화원',
  'A02060900': '도서관',
  'A02061000': '대형서점'},
 'A02_A0207': {'A02070100': '문화관광축제', 'A02070200': '일반축제'},
 'A02_A0208': {},
 'A04_A0401': {},
 'B02_B0201': {'B02010100': '관광호텔',
  'B020102

In [ ]:
cat3_cd2nm = {'A01_A0101': {},
              'A01_A0102': {},
              'A02_A0201': {'A02010100': '고궁',
                            'A02010200': '성',
                            'A02010300': '문',
                            'A02010400': '고택',
                            'A02010500': '생가',
                            'A02010600': '민속마을',
                            'A02010700': '유적지/사적지',
                            'A02010800': '사찰',
                            'A02010900': '종교성지',
                            'A02011000': '안보관광'},
              'A02_A0202': {},
              'A02_A0203': {'A02030100': '농.산.어촌 체험',
                            'A02030200': '전통체험',
                            'A02030300': '산사체험',
                            'A02030400': '이색체험',
                            'A02030500': '관광농원',
                            'A02030600': '이색거리'},
              'A02_A0204': {},
              'A02_A0205': {'A02050100': '다리/대교',
                            'A02050200': '기념탑/기념비/전망대',
                            'A02050300': '분수',
                            'A02050400': '동상',
                            'A02050500': '터널',
                            'A02050600': '유명건물'},
              'A02_A0206': {'A02060100': '박물관',
                            'A02060200': '기념관',
                            'A02060300': '전시관',
                            'A02060400': '컨벤션센터',
                            'A02060500': '미술관/화랑',
                            'A02060600': '공연장',
                            'A02060700': '문화원',
                            'A02060800': '외국문화원',
                            'A02060900': '도서관',
                            'A02061000': '대형서점'},
              'A02_A0207': {'A02070100': '문화관광축제', 'A02070200': '일반축제'},
              'A02_A0208': {},
              'A04_A0401': {},
              'B02_B0201': {'B02010100': '관광호텔',
                            'B02010200': '수상관광호텔',
                            'B02010300': '전통호텔',
                            'B02010400': '가족호텔',
                            'B02010500': '콘도미니엄',
                            'B02010600': '유스호스텔',
                            'B02010700': '펜션',
                            'B02010800': '여관',
                            'B02010900': '모텔',
                            'B02011000': '민박'},
              'C01_C0112': {},
              'C01_C0113': {},
              'C01_C0114': {},
              'C01_C0115': {},
              'C01_C0116': {'C01160001': '캠핑코스'},
              'C01_C0117': {'C01170001': '맛코스'}}

In [26]:
cat_cd_l1 = [] # code list
cat_cd_l2 = [] # code list
cat_cd_l3 = [] # code list
cat_nm_l1 = [] # name list
cat_nm_l2 = [] # name list
cat_nm_l3 = [] # name list
for i in cat3_cd2nm.keys():
#     i
    c = i.split('_')
    c1 = c[0]
    c2 = c[1]
    for j in cat3_cd2nm[i].keys():
#         j
#         cat3_cd2nm[i][j]
        cat_cd_l1.append(c1)
        cat_cd_l2.append(c2)
        cat_cd_l3.append(j)
        cat_nm_l1.append(cat1_cd2nm[c1])
        cat_nm_l2.append(cat2_cd2nm[c1][c2])
        cat_nm_l3.append(cat3_cd2nm[i][j])

In [28]:
len(cat_cd_l1) == len(cat_cd_l2) == len(cat_cd_l3) == len(cat_nm_l1) == len(cat_nm_l2) == len(cat_nm_l3)

True

In [31]:
catinfo = pd.DataFrame({'cat1cd':cat_cd_l1,
                        'cat2cd':cat_cd_l2,
                        'cat3cd':cat_cd_l3,
                        'cat1nm':cat_nm_l1,
                        'cat2nm':cat_nm_l2,
                        'cat3nm':cat_nm_l3})
catinfo.head()

,cat1cd,cat2cd,cat3cd,cat1nm,cat2nm,cat3nm
0,A02,A0201,A02010100,인문(문화/예술/역사),역사관광지,고궁
1,A02,A0201,A02010200,인문(문화/예술/역사),역사관광지,성
2,A02,A0201,A02010300,인문(문화/예술/역사),역사관광지,문
3,A02,A0201,A02010400,인문(문화/예술/역사),역사관광지,고택
4,A02,A0201,A02010500,인문(문화/예술/역사),역사관광지,생가


In [32]:
# catinfo.to_csv('catinfo.csv', index=False)

# #################################### 실행해야할 부분

## ii) csv values를 위한 조회
cat이 cd로 출력되는데 이것은 나중에 mapping으로 바꿔주기로 한다.

In [2]:
CIDs = pd.read_csv('cids.csv')
cids = list(CIDs['cid'].values)

In [3]:
len(cids)

34035

In [7]:
# 각 오퍼레이션 별로 워드에서 검색했을 때, table의 항목구분 열에 1이라고 표시돼있는 필수 파라미터를 다 작성하지 않을 경우 soup 출력시 에러남.

API_HOST = 'http://apis.data.go.kr/B551011/KorService/'
key = '키키키키키키키키키키키키키키키키' # Encoding key

## 오퍼레이션 담은 dic
op = {}
op['det'] = 'detailCommon' # 공통정보조회
op['cat'] = 'categoryCode' # catcd name 조회


# 선택 조회
fimg = 'firstImageYN' # 대표이미지조회
ov = 'overviewYN' # 개요조회
catcd = 'catcodeYN' # 서비스분류코드조회
ar = 'areacodeYN' # 지역코드조회
addr = 'addrinfoYN'# 주소조회
mapin4 = 'mapinfoYN'# 좌표조회



# CID = cids[0]
# # csv values를 위한 조회
# url = f'{API_HOST}{op["det"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&contentId={CID}&defaultYN=Y&{fimg}=Y&{ov}=Y&{catcd}=Y&{ar}=Y&{addr}=Y&{mapin4}=Y'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'xml') # parser는 'lxml'로도 가능.


cnt = 0
df = pd.DataFrame()
for enu, CID in tqdm(enumerate(cids)):
    # p81, content id가 ~~인 공통정보의 '기본정보' '대표이미지' '개요' 조회
    url = f'{API_HOST}{op["det"]}?serviceKey={key}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&contentId={CID}&defaultYN=Y&{fimg}=Y&{ov}=Y&{catcd}=Y&{mapin4}=Y'#&{ar}=Y&{addr}=Y
    response = requests.get(url)
    if (response.status_code == 200) | (response.ok) :
        print('Request OK')
    else:
        print(response.status_code)
    
    soup = BeautifulSoup(response.text, 'xml') # parser는 'lxml'로도 가능.
    
    soup
    
    ## values
    if soup.firstimage.string:
        imgurl1 = soup.firstimage.string
    else:
        imgurl1 = np.NaN

    if soup.firstimage2.string:
        imgurl2 = soup.firstimage2.string
    else:
        imgurl2 = np.NaN
    ovvw = soup.overview.string
    cat1_ = soup.cat1.string
    cat2_ = soup.cat2.string
    cat3_ = soup.cat3.string
    
    ## tmp_df
    tmp_df = pd.DataFrame({'cid' : [CID],
                           'imgurl1' : [imgurl1],
                           'imgurl2' : [imgurl2],
                           'overview' : [ovvw],
                           'cat1' : [cat1_],
                           'cat2' : [cat2_],
                           'cat3' : [cat3_]})
    ## concat & df update
    df = pd.concat([df, tmp_df], ignore_index = True)
    
    cnt
    cnt+=1
#     if enu > 2: # break를 거는 위치가 코드가 다 끝난 다음이니까 2번째까지는 하고 멈추라는 거임.
#         break

0it [00:00, ?it/s]

Request OK


<?xml version="1.0" encoding="utf-8"?>
<OpenAPI_ServiceResponse>
<cmmMsgHeader>
<errMsg>SERVICE ERROR</errMsg>
<returnAuthMsg>HTTP_ERROR</returnAuthMsg>
<returnReasonCode>04</returnReasonCode>
</cmmMsgHeader>
</OpenAPI_ServiceResponse>

0it [01:00, ?it/s]


AttributeError: 'NoneType' object has no attribute 'string'

In [38]:
soup

<?xml version="1.0" encoding="utf-8"?>
<OpenAPI_ServiceResponse>
<cmmMsgHeader>
<errMsg>SERVICE ERROR</errMsg>
<returnAuthMsg>HTTP_ERROR</returnAuthMsg>
<returnReasonCode>04</returnReasonCode>
</cmmMsgHeader>
</OpenAPI_ServiceResponse>

- cid 개수 업데이트 되는 문제
- 여전히 앞쪽 번호는 없는 문제
- cat이 이름이 아닌 id로 출력되는 문제 (catinfo)
- for loop을 계속 돌렸을 때 loop 초반에 자꾸 HTTP error가 발생하는 문제

In [9]:
df

,cid,imgurl1,imgurl2,overview,cat1,cat2,cat3
0,2750144,NaN,NaN,"공주시 원도심에 자리잡은 가가상점은 공주 지역에서 활동하는 예술가 단체, 화가, 공...",A04,A0401,A04010600
1,2805408,NaN,NaN,100%수제돈까스로 만들며 얼리지 않은 1등급 생등심만을 사용하고 있다. 방부제를 ...,A05,A0502,A05020300
2,2750143,NaN,NaN,조용하고 고즈넉한 골목길에 자리잡은 가가책방은 아담하고 포근한 아지트 느낌이 가득한...,A02,A0206,A02061000
3,983886,http://tong.visitkorea.or.kr/cms/resource/85/1...,http://tong.visitkorea.or.kr/cms/resource/85/1...,가가펜션은 절기마다 녹음과 단풍으로 수놓은 중미산 자연휴양림 계곡에 위치하고 있으며...,B02,B0201,B02010700


In [348]:
# for loop을 위한 base
# catid는 for loop에서 받기

if soup.firstimage.string:
    imgurl1 = soup.firstimage.string
else:
    imgurl1 = np.NaN
    
if soup.firstimage2.string:
    imgurl2 = soup.firstimage2.string
else:
    imgurl2 = np.NaN
    
ovvw = soup.overview.string
cat1_ = soup.cat1.string
cat2_ = soup.cat2.string
cat3_ = soup.cat3.string

imgurl1
imgurl2
ovvw
cat1_
cat2_
cat3_

nan

nan

'공주시 원도심에 자리잡은 가가상점은 공주 지역에서 활동하는 예술가 단체, 화가, 공방들이 기획하고 제작한 로컬 굿즈를 판매하고 있으며, 여행자와 방문자를 위한 추천 도서와 관련 상품들도 판매하고 있다.<br>'

'A04'

'A0401'

'A04010600'

# 가장 이상적인 df 결과

In [354]:
df = pd.DataFrame({'imgurl1' : [imgurl1],
'imgurl2' : [imgurl2],
'ovvw' : [ovvw],
'cat1_' : [cat1_],
'cat2_' : [cat2_],
'cat3_' : [cat3_]}, index = [0])
df

,imgurl1,imgurl2,ovvw,cat1_,cat2_,cat3_
0,NaN,NaN,"공주시 원도심에 자리잡은 가가상점은 공주 지역에서 활동하는 예술가 단체, 화가, 공...",A04,A0401,A04010600


## 매칭테스트 2

In [214]:
type(cids.iloc[1].values[0])
cids.iloc[1].values[0]

125405

In [216]:
'0000059' in CIDs
sorted(intCIDs)[-10:] # [:10]# , reverse = True

[2832111,
 2832112,
 2832118,
 2832133,
 2832135,
 2832136,
 2832140,
 2832154,
 2832160,
 2832164]